# Quora insincere questions

### Objective of the competition : predict if a question is insincere (1) or not (0) to help the moderation system of Quora. 
**Parameters to evaluate the insincerity:**
* non-neutral tone (exagerated or rhetorical)
* disparaging or inflamatory questions (suggest discriminatory idea, hateful, based on prejudices)
* not grounded in reality (absurd or false)
* sexual content for shock value + illicite sexual content

I'm new to machine learning so i began by doing a topic modeling with gensim to see if i can identify recurent topics differencing sincere and insincere question.

The machine learning part is a more concise cause i wanted to focus on topic modeling. 

# I. Imports and loading the data

In [ ]:
import os

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

In [ ]:
#from vivadata.datasets.common import get_path_for_dataset
#base_path = get_path_for_dataset('quora')
X_train_filepath = os.path.join('..', 'input', 'train.csv')
X_test_filepath = os.path.join('..', 'input', 'test.csv')
sample_filepath = os.path.join('..', 'input', 'sample_submission.csv')
X_train_filepath, X_test_filepath, sample_filepath

In [ ]:
df_train = pd.read_csv(X_train_filepath)
df_test = pd.read_csv(X_test_filepath)
sample = pd.read_csv(sample_filepath)
df_train.shape, df_test.shape, sample.shape

Now that the data is loaded, let's take a quick look. 
![](https://media.giphy.com/media/94hqi5hBHu5W/giphy.gif)

#  II.EDA

In [ ]:
df_train.sample(10)

- **qid** = unique question identifier
- **question_text** = text of the question
- **target** = 1: insincere question, 0:not insincere question

In [ ]:
df_test.sample(10)

**qid** = unique question identifier
**question_text** = text of the question

In [ ]:
df_train.info()

In [ ]:
df_train.describe()

In [ ]:
ax, fig = plt.subplots(figsize=(10, 7))
sns.countplot(x='target', data=df_train)
plt.title('Reparition of question by insincerity');

There is 80 810 insincere questions on 1 306 122, so around 6% (0.0618). This would be a problem for the machin learning process cause it's an important disparity with 1 225 312 sincere ones. 

**_Now, we need to transform the data from text to something the computer could understand in order to study it more precisely._**

# III.Topic modelisation

I'm gonna prepare the data,  then use gensim and lda modeling to create topics for sincere and insincere questions. 

In [ ]:
import gensim
import nltk
from nltk.corpus import stopwords

In [ ]:
# Define the target and the variable.
y_train = df_train.loc[:, 'target']
X_train = df_train.loc[:, 'question_text']
X_train.shape, y_train.shape

In [ ]:
X_train.head()

In [ ]:
# Create a variable with all the sincere questions.
X_train_sincere = X_train[df_train['target'] == 0]
X_train_sincere[:5]

In [ ]:
# Create a variable with all insincere questions.
X_train_insincere = X_train[df_train['target'] == 1]
X_train_insincere[:5]

## III-A. Topic modeling with 10 topics

### III-A. 1. Tokenisation

We need to transform the text data in token to be able to compute it. 

In [ ]:
# First i load the list of stopwords, the words which aren't useful to understand the meaning.
stop_words = stopwords.words('english')
stop_words[:10]

In [ ]:
# I transform the sincere questions in a list of words.
sincere_prepro_questions = [gensim.utils.simple_preprocess(question) 
                            for question in X_train_sincere]
sincere_prepro_questions[:2]

In [ ]:
# I transform the insincere questions in a list of words.
insincere_prepro_questions = [gensim.utils.simple_preprocess(question) 
                              for question in X_train_insincere]
insincere_prepro_questions[:2]

In [ ]:
# Verifying the length of the both lists.
len(sincere_prepro_questions), len(insincere_prepro_questions)

In [ ]:
# I remove the stopwords from the sincere questions list of words.
clear_sincere_questions = [[word for word in question if word not in stop_words] 
                             for question in sincere_prepro_questions]
clear_sincere_questions[:2]

In [ ]:
# I do the same for insincere one.
clear_insincere_questions = [[word for word in question if word not in stop_words] 
                             for question in insincere_prepro_questions]
clear_insincere_questions[:2]

### III-A.2. Creation of gensim dictionnaries

In [ ]:
# Creation of a dictionnary of the words and their id for sincere questions
sincere_questions_dictionary = gensim.corpora.Dictionary(clear_sincere_questions)
sincere_token = sincere_questions_dictionary.token2id

In [ ]:
# Association of the dictonary words and their frequency for sincere questions
sincere_dict_frequency = {sincere_questions_dictionary[k]: v for k,v in sincere_questions_dictionary.dfs.items()}

In [ ]:
# Creation of a dictionnary of the words and their id for insincere questions
insincere_questions_dictionary = gensim.corpora.Dictionary(clear_insincere_questions)
sincere_token = sincere_questions_dictionary.token2id

In [ ]:
# Association of the dictonary words and their frequency for insincere questions
sincere_dict_frequency = {sincere_questions_dictionary[k]: v for k,v in sincere_questions_dictionary.dfs.items()}

### III-A.3. Vectorization

In [ ]:
# I transform the elements of the sincere dictionary in vectors. 
sincere_corpus = [sincere_questions_dictionary.doc2bow(question) 
                  for question in clear_sincere_questions]
sincere_corpus[:2]

In [ ]:
# I do the same for the insincere questions. 
insincere_corpus = [insincere_questions_dictionary.doc2bow(question) 
                    for question in clear_insincere_questions]
insincere_corpus[:2]

### III-A.4. Lda Modeling

Lda modeling is a statistical modeling which discovers abstract "topics" in documents. Here Lda modeling will help me to see if sincere and insincere questions have some majors topics. 

This topics could be used as a variable later, for the machine learning.

In [ ]:
# Creating a lda model, which create 10 topics from the sincere questions
lda_model_sincere = gensim.models.ldamodel.LdaModel(
    corpus=sincere_corpus, num_topics=10, id2word=sincere_questions_dictionary,
    random_state=25, passes=5)

In [ ]:
# Creating a lda model, which create 10 topics from the insincere questions
lda_model_insincere = gensim.models.ldamodel.LdaModel(
    corpus=insincere_corpus, num_topics=10, id2word=insincere_questions_dictionary,
    random_state=25, passes=5)

In [ ]:
from pprint import pprint

In [ ]:
# Printing the result of the modeling for sincere questions.
pprint(lda_model_sincere.print_topics())

In [ ]:
# Same for insincere ones. 
pprint(lda_model_insincere.print_topics())

### III-A.5. Visualization

I will now measure how good the topic modeling is and create a visual of the topics. 

Measurement of how good a topic model is. 
* Coherence score : 
    * intrinsic measure: compare a word with the preceding & sucseeding ones. Calculate a log probability
    * extrinsic measure: every single word is paired with every others. Use pointwise mutual information. 

In [ ]:
import pyLDAvis.gensim
pyLDAvis.enable_notebook()

In [ ]:
pyLDAvis.gensim.prepare(lda_model_sincere, sincere_corpus, sincere_questions_dictionary)

Some topics overlapt each other (1 &4, 9&7, 2&3). The topic are less easily recognizable than for insincere questions. We can 
recognize word showing interogation, some vocabulary about computer... but the vocabulary is larger and more miscellaneous. 

In [ ]:
pyLDAvis.gensim.prepare(lda_model_insincere, insincere_corpus, 
                        insincere_questions_dictionary)

The number of topics are not optimal, we can see that some topics (1, 3, 7 and 2, 5) overlapt. Maybe those subjects could
could have be reunited in a same topics. 

The 1, 3, 7 seems to be about politic:
    * the first about Donald Trump;
    * the third about international politic;
    * the seventh about political parties. 
 We also see that the 30 most relevant words are sometimes repeated in the 3 topics ('trump', 'liberals', 'president', 'supporters'). 

The 2, 5 topics seems to be about racial and religious subject:
    * the second refers to race ('white', 'black', 'chinese');
    * and the second seems to be about Islam ('muslims', 'muslim', 'pakistant'). 
   
Topics 8 and 6, even if they are not close to the 1st cluster contain a lot of similar words and seems to also be about politics:
       * the 8th about Donald Trump's election also ('trump', 'president', 'obama')
       * the 6th about interior politic and social subjects ('gun', 'clinton', 'rape')
       
The 9th topic is about religion ('jews', 'christians', 'atheist')

The 10th and 4th seems to be about sex and gender:
    * the 4th more about sex ('girl', 'sex', 'sexual');
    * the 10th is less easy to classify but seems to be more about gender ('girl', 'gender', 'bible')

All those subjects has in common to be really polemics and suitable to trolls. This criteria would be easily used by a human 
control but we need to find how to train your model to try to recognize it. 

The insincere questions seems easier to distinct than the sincere ones, cause they are centred on a less various spectrum of words. 

---

**Now we do the same process with bigrams!**!

## III-B. Topics modeling with bigrams (10 topics)

### III-B.1. Creation of the bigrams (from preprocessed data)

In [ ]:
from gensim.models import Phrases
from gensim.models.phrases import Phraser

In [ ]:
# Create the bigram for sincere questions.
sincere_bigram_model = Phrases(sincere_prepro_questions, min_count=1, threshold=2)

In [ ]:
# Same for insincere ones. 
insincere_bigram_model = Phrases(insincere_prepro_questions, min_count=1, threshold=2)

In [ ]:
# Creation of the phrasers, which is needed to use the bigrams. First for sincere questions...
sincere_bigram_phraser = Phraser(sincere_bigram_model)

In [ ]:
# Then for the insincere. 
insincere_bigram_phraser = Phraser(sincere_bigram_model)

### III-B.2.Creation of the bigram dictionary

In [ ]:
sincere_bigrams = [sincere_bigram_model[question] for question in sincere_prepro_questions]
sincere_bigrams[:2]

In [ ]:
insincere_bigrams = [insincere_bigram_model[question] for question in insincere_prepro_questions]
insincere_bigrams[:2]

In [ ]:
# Creation of a dictionary with the words and their id for sincere bigrams
sincere_bigrams_dictionary = gensim.corpora.Dictionary(sincere_bigrams)
sincere_bigrams_token = sincere_bigrams_dictionary.token2id

In [ ]:
# Association of the dictonary words and their frequency for sincere bigrams
sincere_bigrams_frequency = {sincere_bigrams_dictionary[k]: 
                             v for k,v in sincere_bigrams_dictionary.dfs.items()}

In [ ]:
# Creation of a dictionary with the words and their id for insincere bigrams
insincere_bigrams_dictionary = gensim.corpora.Dictionary(insincere_bigrams)
insincere_bigrams_token = insincere_bigrams_dictionary.token2id

In [ ]:
# Association of the dictonary words and their frequency for insincere bigrams
insincere_bigrams_frequency = {insincere_bigrams_dictionary[k]: 
                               v for k,v in insincere_bigrams_dictionary.dfs.items()}

### III-B.3. Vectorization

In [ ]:
sincere_bigrams_corpus = [sincere_bigrams_dictionary.doc2bow(question) 
                  for question in clear_sincere_questions]
sincere_bigrams_corpus[:2]

In [ ]:
insincere_bigrams_corpus = [insincere_bigrams_dictionary.doc2bow(question) 
                  for question in clear_insincere_questions]
insincere_bigrams_corpus[:2]

 ### III-B.4. Lda Modeling

In [ ]:
# Creating a lda model, which create 10 topics from the sincere bigrams
lda_model_sincere_bigrams = gensim.models.ldamodel.LdaModel(
    corpus=sincere_bigrams_corpus, num_topics=10, id2word=sincere_bigrams_dictionary,
    random_state=25, passes=5)

In [ ]:
# Creating a lda model, which create 10 topics from the insincere bigrams
import warnings
warnings.filterwarnings('ignore')

lda_model_insincere_bigrams = gensim.models.ldamodel.LdaModel(
    corpus=insincere_bigrams_corpus, num_topics=10, id2word=insincere_bigrams_dictionary,
    random_state=25, passes=5)

In [ ]:
pprint(lda_model_sincere_bigrams.print_topics())

In [ ]:
pprint(lda_model_insincere_bigrams.print_topics())

### III-B.5. Computing cohenrence and visualization

In [ ]:
coherence_lda_model_sincere_b = CoherenceModel(
    model=lda_model_sincere_bigrams, texts=clear_sincere_questions, 
    dictionary=sincere_bigrams_dictionary, coherence='c_v')

coherence_lda_3 = coherence_lda_model_sincere_b.get_coherence()
coherence_lda_3

In [ ]:
pyLDAvis.gensim.prepare(lda_model_sincere_bigrams, sincere_bigrams_corpus, 
                        sincere_bigrams_dictionary)

Even if the graph shows unique word, the circles take into account the bigrams. 

The repartition of the topics is quite different and some emerge more clearly. 

The first topic, which is one of the 2 biggest, seems to be about education and job. 

The 2nd one, which is also quite big, gather word indicating interrogation. It seems to be less a topic and more markers of real question (opposed to affirmation disguise in question)

The others topics are more mixed and once again some of them overlap each other, indicating that 10 topics is probably too much. Some secondary subjects appears but they're not as recognizable as insincere questions. 


In [ ]:
coherence_lda_model_insincere_b = CoherenceModel(
    model=lda_model_insincere_bigrams, texts=clear_insincere_questions, 
    dictionary=insincere_bigrams_dictionary, coherence='c_v')

coherence_lda_4 = coherence_lda_model_insincere_b.get_coherence()
coherence_lda_4

In [ ]:
pyLDAvis.gensim.prepare(lda_model_insincere_bigrams, insincere_bigrams_corpus, 
                        insincere_bigrams_dictionary)

The result are quite similar to the first one. Even if some subjects are analogous we can recognize most of the topics by reading the 
most relevant words. 
* Topics 1, 4 and 8 are about politics.
*  Topics 2, 3 and  6, are about race and religions.
* Topics 5, 7 and 9 are sex questions
*  Topic 10 is more contrasted and we couldn't recognize a clear subject, it seems to gather different hot social subjects (gender, guns, criminals)

_**The clear separation in 4 group of subject let think we should try a lda with 4 topics. **_

## III-C. Recreate new Lda with 4 topics

We already have preprocessed data, wo we will just recreate the Lda in itself. 

### III-C.1. Creating lda model

In [43]:
# Creating a lda model, which create 4 topics from the sincere questions
import warnings
warnings.filterwarnings('ignore')
lda_model_sincere_4 = gensim.models.ldamodel.LdaModel(
    corpus=sincere_corpus, num_topics=4, id2word=sincere_questions_dictionary,
    random_state=25, passes=5)

In [41]:
# Creating a lda model, which create 4 topics from the insincere questions
lda_model_insincere_4 = gensim.models.ldamodel.LdaModel(
    corpus=insincere_corpus, num_topics=4, id2word=insincere_questions_dictionary,
    random_state=25, passes=5)

In [44]:
# Printing the result of the modeling for sincere questions.
pprint(lda_model_sincere_4.print_topics())

[(0,
  '0.021*"would" + 0.008*"us" + 0.008*"world" + 0.008*"people" + 0.007*"like" '
  '+ 0.007*"think" + 0.006*"india" + 0.006*"old" + 0.006*"country" + '
  '0.005*"many"'),
 (1,
  '0.014*"like" + 0.013*"people" + 0.010*"someone" + 0.010*"life" + '
  '0.009*"best" + 0.009*"would" + 0.009*"get" + 0.009*"know" + 0.008*"ever" + '
  '0.008*"one"'),
 (2,
  '0.009*"much" + 0.008*"time" + 0.006*"day" + 0.005*"used" + 0.005*"use" + '
  '0.004*"system" + 0.004*"take" + 0.004*"long" + 0.004*"class" + '
  '0.004*"water"'),
 (3,
  '0.020*"best" + 0.019*"get" + 0.009*"good" + 0.007*"india" + 0.006*"job" + '
  '0.006*"quora" + 0.005*"college" + 0.005*"engineering" + 0.005*"online" + '
  '0.005*"university"')]


In [42]:
# Printing the result of the modeling for insincere questions.
pprint(lda_model_insincere_4.print_topics())

[(0,
  '0.037*"people" + 0.015*"white" + 0.012*"world" + 0.011*"americans" + '
  '0.010*"chinese" + 0.010*"black" + 0.009*"country" + 0.009*"us" + '
  '0.008*"muslims" + 0.008*"hate"'),
 (1,
  '0.032*"women" + 0.023*"men" + 0.015*"girls" + 0.014*"sex" + 0.013*"like" + '
  '0.010*"get" + 0.008*"would" + 0.008*"gay" + 0.008*"indian" + 0.008*"man"'),
 (2,
  '0.024*"people" + 0.020*"quora" + 0.011*"muslims" + 0.010*"india" + '
  '0.008*"questions" + 0.008*"god" + 0.008*"like" + 0.008*"many" + '
  '0.007*"indians" + 0.007*"christians"'),
 (3,
  '0.031*"trump" + 0.010*"liberals" + 0.008*"president" + 0.008*"would" + '
  '0.007*"obama" + 0.007*"donald" + 0.007*"democrats" + 0.006*"us" + '
  '0.005*"people" + 0.005*"right"')]


### III-C.2. Visualization

In [50]:
# Display the topic modeling for sincere questions.
pyLDAvis.gensim.prepare(lda_model_sincere_4, sincere_corpus, sincere_questions_dictionary)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
3      0.163952 -0.261155       1        1  27.851519
1     -0.136546 -0.143456       2        1  27.332216
0     -0.282503  0.148785       3        1  23.342737
2      0.255096  0.255827       4        1  21.473526, topic_info=     Category          Freq   ...     loglift  logprob
term                          ...                     
11    Default  56145.000000   ...     30.0000  30.0000
462   Default  61552.000000   ...     29.0000  29.0000
64    Default  57738.000000   ...     28.0000  28.0000
345   Default  41514.000000   ...     27.0000  27.0000
9     Default  40818.000000   ...     26.0000  26.0000
269   Default  21168.000000   ...     25.0000  25.0000
141   Default  21163.000000   ...     24.0000  24.0000
104   Default  17832.000000   ...     23.0000  23.0000
482   Default  14684.000000   ...     22.0000  22.0000
61    Default  16279.000000   ...     21.0000  21.0000
297   Default  14512.000000   ...     20.0000  20.0000
1772  Default  15250.000000   ...     19.0000  19.0000
1124  Default  13696.000000   ...     18.0000  18.0000
15    Default  21825.000000   ...     17.0000  17.0000
62    Default  13338.000000   ...     16.0000  16.0000
339   Default  21083.000000   ...     15.0000  15.0000
226   Default  12519.000000   ...     14.0000  14.0000
125   Default  22360.000000   ...     13.0000  13.0000
42    Default  12172.000000   ...     12.0000  12.0000
644   Default   9713.000000   ...     11.0000  11.0000
558   Default  10088.000000   ...     10.0000  10.0000
1837  Default   9778.000000   ...      9.0000   9.0000
277   Default  10797.000000   ...      8.0000   8.0000
244   Default  10223.000000   ...      7.0000   7.0000
72    Default  10053.000000   ...      6.0000   6.0000
1258  Default   9862.000000   ...      5.0000   5.0000
249   Default   9807.000000   ...      4.0000   4.0000
99    Default   9582.000000   ...      3.0000   3.0000
261   Default   9533.000000   ...      2.0000   2.0000
85    Default   8418.000000   ...      1.0000   1.0000
...       ...           ...   ...         ...      ...
1349   Topic4   3954.963135   ...      1.5382  -6.0203
2459   Topic4   4085.754883   ...      1.5382  -5.9878
1728   Topic4   3768.897949   ...      1.5381  -6.0685
662    Topic4   3659.785889   ...      1.5381  -6.0979
1274   Topic4   3622.738281   ...      1.5381  -6.1080
848    Topic4   3496.422119   ...      1.5381  -6.1435
1764   Topic4   3525.195312   ...      1.5381  -6.1353
3203   Topic4   3429.128174   ...      1.5381  -6.1630
14     Topic4   3434.766602   ...      1.5381  -6.1613
3415   Topic4   3297.162109   ...      1.5381  -6.2022
4840   Topic4   3341.036865   ...      1.5381  -6.1890
1790   Topic4   3260.350342   ...      1.5381  -6.2134
940    Topic4   3154.689453   ...      1.5381  -6.2464
3834   Topic4   3221.226318   ...      1.5381  -6.2255
339    Topic4  14791.953125   ...      1.1840  -4.7012
15     Topic4  12774.125000   ...      1.0027  -4.8479
587    Topic4   7180.004395   ...      1.1237  -5.4240
21     Topic4   8012.076660   ...      1.0489  -5.3143
113    Topic4   7201.226562   ...      0.6841  -5.4210
94     Topic4   7607.585938   ...      0.5701  -5.3661
504    Topic4   6015.426270   ...      0.8360  -5.6009
341    Topic4   6231.890137   ...      0.4635  -5.5656
129    Topic4   6470.606934   ...      0.3626  -5.5280
298    Topic4   4692.453125   ...      0.9738  -5.8493
137    Topic4   5099.904785   ...      0.4292  -5.7661
110    Topic4   5888.275391   ...     -0.3112  -5.6223
282    Topic4   5703.202148   ...     -0.2124  -5.6542
160    Topic4   4896.874512   ...     -0.1003  -5.8067
51     Topic4   4479.308105   ...      0.6134  -5.8958
1782   Topic4   4319.733887   ...      0.8909  -5.9321

[208 rows x 6 columns], token_table=      Topic      Freq      Term
term                           
449       1  0.999872   account
12        3  0.999780    affect
127

The 4 topics created by the lda model are very distinct, this is better than the test with 10 topics. 
We still can't differenciate clearly the subjects of the topics. Some words come back a lot "best", "guess", "world", "people".
This tend to show that the sincere question have various subject and there is no common theme around them. 

In [48]:
# # Display the topic modeling for insincere questions.
pyLDAvis.gensim.prepare(lda_model_insincere_4, insincere_corpus, insincere_questions_dictionary)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
3      0.029690  0.239387       1        1  27.428501
0      0.163631  0.026489       2        1  26.352978
1     -0.296509 -0.040013       3        1  23.656755
2      0.103187 -0.225863       4        1  22.561762, topic_info=     Category          Freq       Term         Total  loglift  logprob
term                                                                  
278   Default   5536.000000      women   5536.000000  30.0000  30.0000
122   Default   6078.000000      trump   6078.000000  29.0000  29.0000
136   Default  12113.000000     people  12113.000000  28.0000  28.0000
393   Default   3926.000000        men   3926.000000  27.0000  27.0000
191   Default   3237.000000      quora   3237.000000  26.0000  26.0000
249   Default   2505.000000      girls   2505.000000  25.0000  25.0000
194   Default   2360.000000        sex   2360.000000  24.0000  24.0000
5     Default   2338.000000      world   2338.000000  23.0000  23.0000
515   Default   3581.000000      white   3581.000000  22.0000  22.0000
171   Default   2051.000000   liberals   2051.000000  21.0000  21.0000
237   Default   1968.000000    chinese   1968.000000  20.0000  20.0000
197   Default   3345.000000    muslims   3345.000000  19.0000  19.0000
367   Default   1733.000000    country   1733.000000  18.0000  18.0000
176   Default   3039.000000      india   3039.000000  17.0000  17.0000
91    Default   2714.000000  americans   2714.000000  16.0000  16.0000
418   Default   1389.000000  questions   1389.000000  15.0000  15.0000
130   Default   1594.000000  president   1594.000000  14.0000  14.0000
510   Default   2642.000000      black   2642.000000  13.0000  13.0000
83    Default   1325.000000        god   1325.000000  12.0000  12.0000
25    Default   1364.000000        gay   1364.000000  11.0000  11.0000
1018  Default   1311.000000        man   1311.000000  10.0000  10.0000
263   Default   2853.000000         us   2853.000000   9.0000   9.0000
570   Default   1415.000000      obama   1415.000000   8.0000   8.0000
668   Default   1403.000000     donald   1403.000000   7.0000   7.0000
47    Default   1345.000000      china   1345.000000   6.0000   6.0000
45    Default   1238.000000      woman   1238.000000   5.0000   5.0000
139   Default   1338.000000  countries   1338.000000   4.0000   4.0000
231   Default   2444.000000    indians   2444.000000   3.0000   3.0000
680   Default   1351.000000  democrats   1351.000000   2.0000   2.0000
1848  Default   1269.000000     racist   1269.000000   1.0000   1.0000
...       ...           ...        ...           ...      ...      ...
3775   Topic4    356.560089      tamil    357.287598   1.4869  -6.1281
97     Topic4    328.299042   language    329.027985   1.4867  -6.2107
434    Topic4    324.709229    english    325.437225   1.4867  -6.2217
398    Topic4    317.481323    atheist    318.207520   1.4866  -6.2442
2675   Topic4    318.730835      bible    319.459900   1.4866  -6.2402
716    Topic4    320.231506       dumb    320.972748   1.4866  -6.2356
101    Topic4    314.564545      speak    315.298187   1.4866  -6.2534
1312   Topic4    312.723450       hell    313.458313   1.4866  -6.2593
436    Topic4    298.746735      hindi    299.465820   1.4865  -6.3050
54     Topic4    302.737579   superior    303.484070   1.4865  -6.2917
4050   Topic4    318.306610       body    319.091949   1.4864  -6.2416
4305   Topic4    346.158905      might    347.024963   1.4864  -6.1577
246    Topic4    796.351685   religion    799.294617   1.4852  -5.3246
197    Topic4   1830.795898    muslims   3345.316895   0.8861  -4.4921
176    Topic4   1611.784424      india   3039.543457   0.8545  -4.6195
136    Topic4   3970.960205     people  12113.837891   0.3736  -3.7178
652    Topic4    709.376465      islam   1074.548096   1.0736  -5.4402
231    Topic4   1132.744263    indians   2444.744629   0.7196  -4.9722
350    Topic

We could see 4 distincts topics: 
- topic 1 is about politic with top-3 keyword being "Trump", "liberals", "presidents". It's also the bigger topic;
- topic 2 is about race and nationality with keywords "white", "black", "people";
- topic 3 is about gender and sex, "women", "men", "gay";
- topic 4 is about religion it seems, with keywords "christians", "muslims".

This topics could be use cause they rejoign the subjects given by Quora as sensible. This topic could be used as a variable 
to characteristic insincere question and be used for machine learning. 

---

#### Quick machine learning model

This kernel is mostly center around the topic modeling but let's try a quick and simple model just to see how it works. This model doesn't use the topic modeling, which maybe could help to improve the final score. 

In [51]:
from sklearn.metrics import classification_report
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import make_pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

Train_test_split is not the most precise method of validation but I just want to quickly test the model as a first try. 

In [52]:
X_train, X_test, y_train, y_test = train_test_split(X_train, y_train)
X_train.shape, y_train.shape, X_test.shape, y_test.shape

((979591,), (979591,), (326531,), (326531,))

Now i have my training and testing sample for train my model. I will do a pipeline with a transformer, to turn the text value to vectors and an estimator, to predict. 

In [53]:
tfidv = TfidfVectorizer(lowercase=True, stop_words='english')
multinomialnb = MultinomialNB()
pipe = make_pipeline(tfidv, multinomialnb)
pipe

Pipeline(memory=None,
     steps=[('tfidfvectorizer', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth...   vocabulary=None)), ('multinomialnb', MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))])

In [ ]:
pipe.fit(X_train, y_train)
y_pred = pipe.predict(X_test)
print(classification_report(y_test, y_pred))

**As expected for a naive model and without any ponderation on a disbalance datasets, the F1 score is quite good for 0 and really bad for 1.**

**Enginere featuring and ponderation could let us have a better score. **